In [11]:
# !pip install requests pandas matplotlib sanpy
!pip install requests sanpy

In [12]:
import numpy as np
import requests
import pandas as pd
import san
from datetime import date, timedelta

### Отримання даних (Santiment API)

Santiment Crypto API — це аналітичний API, який надає доступ до on-chain, соціальних та ринкових даних криптовалют.
З його допомогою можна отримувати метрики про активність адрес, обсяги транзакцій, поведінку “китів”, ринкову капіталізацію, а також соціальні сигнали для глибшого аналізу ринку.


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
from getpass import getpass
san.ApiConfig.api_key = getpass("Paste your Santiment API key: ")

Paste your Santiment API key: ··········


In [15]:
# --- Test connection---
start_date = '2025-10-01'
end_date = date.today().strftime('%Y-%m-%d')

df = san.get("daily_active_addresses",
    slug="bitcoin",
    from_date=start_date,
    to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
    interval="1d")
df.head(5)

,value
datetime,
2025-10-01 00:00:00+00:00,869490.0
2025-10-02 00:00:00+00:00,806100.0
2025-10-03 00:00:00+00:00,794474.0
2025-10-04 00:00:00+00:00,754199.0
2025-10-05 00:00:00+00:00,691410.0


In [16]:
# --- Налаштування дат ---
start_date = '2024-01-01'
end_date = date.today().strftime('%Y-%m-%d')

# --- Список монет для аналізу ---
coins = ["bitcoin", "ethereum"]

In [20]:
# --- Порожній список для збереження даних усіх монет ---
all_data = []

# --- Цикл по кожній монеті ---
for coin in coins:
    print(f"🔄 Завантаження даних для {coin}...")

    # Metric 1 — Price, USD
    df_price = san.get(
        "price_usd",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 2 — Daily Active Addresses
    df_daa = san.get(
        "daily_active_addresses",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 3 — Transaction Volume
    df_tw = san.get(
        "transaction_volume",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 4 — Transaction Volume
    df_mc = san.get(
        "marketcap_usd",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 5 — Whale Transaction Count > $100k
    df_whale_count_100k = san.get(
        "whale_transaction_count_100k_usd_to_inf",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 6 — Whale Transaction Volume > $100k
    df_whale_volume_100k = san.get(
        "whale_transaction_volume_100k_usd_to_inf",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Metric 7 — Whale Transaction Count > $1m
    df_whale_count_1m = san.get(
        "whale_transaction_count_1m_usd_to_inf",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # Метрика 8 — Whale Transaction Volume > $1m
    df_whale_volume_1m = san.get(
        "whale_transaction_volume_1m_usd_to_inf",
        slug=coin,
        from_date=start_date,
        to_date=(date.today() - timedelta(days=1)).strftime('%Y-%m-%d'),
        interval="1d"
    )

    # --- Формування єдиного датафрейму для монет ---
    df_coin = pd.DataFrame({
        'date': pd.to_datetime(df_daa.index).date,
        'coin_id': coin,
        'price_usd' : df_price['value'],
        'daily_active_addresses': df_daa['value'],
        'transaction_volume': df_tw['value'],
        'marketcap_usd': df_mc['value'],
        'whale_trans_count_100k': df_whale_count_100k['value'],
        'whale_trans_volume_100k': df_whale_volume_100k['value'],
        'whale_trans_count_1m': df_whale_count_1m['value'],
        'whale_trans_volume_1m': df_whale_volume_1m['value']
    }).reset_index(drop=True)

    all_data.append(df_coin)
    print(f"✅ Дані для {coin} додано ({len(df_coin)} рядків)")

# ---  Об’єднання всіх монет ---
santiment_data = pd.concat(all_data, ignore_index=True)

# ---  Перевірка результату ---
display(santiment_data)
print(f"📊 Усього рядків: {len(santiment_data)}")

# ---  Збереження у CSV ---
santiment_data.to_csv("/content/drive/MyDrive/DA_in_IT/crypto_whale_analysis/data/santiment_whale_activity_2024-2025.csv", index=False)
print("💾 Файл збережено як santiment_whale_activity.csv")


🔄 Завантаження даних для bitcoin...
✅ Дані для bitcoin додано (657 рядків)
🔄 Завантаження даних для ethereum...
✅ Дані для ethereum додано (657 рядків)


,date,coin_id,price_usd,daily_active_addresses,transaction_volume,marketcap_usd,whale_trans_count_100k,whale_trans_volume_100k,whale_trans_count_1m,whale_trans_volume_1m
0,2024-01-01,bitcoin,44167.330214,831736.0,2.091889e+05,8.651060e+11,6019.0,8.016904e+09,1388.0,6.471025e+09
1,2024-01-02,bitcoin,44951.138005,873732.0,3.548550e+05,8.804978e+11,10214.0,1.471860e+10,2482.0,1.205896e+10
2,2024-01-03,bitcoin,42848.175641,954003.0,3.549216e+05,8.393459e+11,10996.0,1.417530e+10,2267.0,1.107391e+10
3,2024-01-04,bitcoin,44200.715873,1016355.0,4.251789e+05,8.658898e+11,18409.0,1.708048e+10,2681.0,1.227918e+10
4,2024-01-05,bitcoin,44160.760983,865666.0,3.638873e+05,8.651449e+11,16465.0,1.458675e+10,2157.0,1.053015e+10
...,...,...,...,...,...,...,...,...,...,...
1309,2025-10-14,ethereum,4125.015769,563466.0,5.153881e+06,4.978851e+11,13976.0,1.891606e+10,2692.0,1.588095e+10
1310,2025-10-15,ethereum,3986.398331,643052.0,3.921075e+06,4.811537e+11,12235.0,1.421187e+10,1998.0,1.181621e+10
1311,2025-10-16,ethereum,3894.258843,594972.0,3.200166e+06,4.700323e+11,10816.0,1.120132e+10,1868.0,9.037693e+09
1312,2025-10-17,ethereum,3833.009410,611774.0,4.092808e+06,4.626392e+11,10497.0,1.397764e+10,1823.0,1.186265e+10


📊 Усього рядків: 1314
💾 Файл збережено як santiment_whale_activity.csv
